In [ ]:
import os
os.chdir('./..')

import sys
import time
import concurrent.futures
import pickle

import Methods.common as common
import Methods.dataLoader as dataloader
import Methods.commitLoader as commitloader

try:
    import argparse
    import magic
except ImportError as err:
    print (err)
    sys.exit(-1)

In [ ]:
dataset_file = 'sorted_dataset_opt.pkl'
token_file = '/mnt/c/Users/User1/Desktop/tokens.txt'

In [ ]:
with open(dataset_file, "rb") as f:
    dataset = pickle.load(f)

token_list = list()
with open(token_file) as f:
    lines = f.readlines()
    for line in lines[0].split(','):
        token_list.append(line)
        
lenTokens = len(token_list)

try:
    common.magic_cookie = magic.open(magic.MAGIC_MIME)
    common.magic_cookie.load()
except AttributeError:
    common.magic_cookie = magic.Magic(mime=True, uncompress=True)
common.verbose_print('[-] initialized magic cookie\n')

In [ ]:
len(dataset)

In [ ]:
def getDataUsingThreads(params):
    source, destination, prs, cut_off_date, task, begin, end = params
    start = time.perf_counter()
    
    ct = 0
    
    working_repos = []
    for pair in range(begin, end):
        print(pair, '-', source[pair], '-', destination[pair])
        common.verbose_mode = False

        try:
            destination_sha, ct = dataloader.getDestinationSha(destination[pair], cut_off_date[pair], token_list, ct)
            ct, pr_data, req, runtime = dataloader.fetchPrData(source[pair], destination[pair], prs[pair], destination_sha, token_list, ct)
            with open('Repos_prs_opt/' + str(pair) + '_' + source[pair].split('/')[0] + '_' + source[pair].split('/')[1] + '_prs.pkl', 'wb') as f:
                pickle.dump([pr_data, req, runtime], f)
            working_repos.append(pair)
        except Exception as e:
            print('Exception  :(\n ',e)
    print(working_repos)        
    finish = time.perf_counter()
    return f'Task {task} done executing in ... {round((finish - start) / 60, 2)} minutes'

In [ ]:
def mainThread():
    source = []
    destination = []
    prs = []
    cut_off_date = []
    
    for i in dataset:
        source.append(dataset[i]['source'])
        destination.append(dataset[i]['destination'])
        prs.append(str(dataset[i]['pr']).split('/'))
        cut_off_date.append(dataset[i]['cut_off_date'])
    
    task = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
#     start1 = 0
#     end1 = 1
#     start2 = 1
#     end2 = 3
#     start6 = 3
#     end6 = 7
#     start7 = 7
#     end7 = 10
#     start3 = 10
#     end3 = 15
#     start4 = 15
#     end4 = 31
#     start5 = 45
#     end5 = 66
#     start9 = 48
#     end9 = 50
#     start10 = 150
#     end10 = 342
    start10 = 0
    end10 = 1
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        params = [
#                   (source, destination, prs, cut_off_date, task[0], start1, end1),
#                   (source, destination, prs, cut_off_date, task[1], start2, end2),
#                   (source, destination, prs, cut_off_date, task[2], start3, end3),
#                   (source, destination, prs, cut_off_date, task[3], start4, end4),
#                   (source, destination, prs, cut_off_date, task[4], start5, end5)
#                   (source, destination, prs, cut_off_date, task[5], start6, end6),
#                   (source, destination, prs, cut_off_date, task[6], start7, end7)
#                   (source, destination, prs, cut_off_date, task[7], start8, end8)
#                   (source, destination, prs, cut_off_date, task[8], start9, end9),
                  (source, destination, prs, cut_off_date, task[9], start10, end10)
                 ]
        results = executor.map(getDataUsingThreads, params)

        for result in results:
            print(result)

mainThread()

## To get only the prs used in manual check

In [ ]:
pairs = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,6,6,6,6,6,6,6,6,6,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,10,10,10,11,11,11,11,11,11,11,11,11,11,12,12,12,12,12,12,12,13,13,13,13,13,13,13,13,14,15,15,15,15,15,16,16,16,16,16,16,18,18,19,20,23,23,23,24,24,24,25,25,26,26,26,26,26,26,27,27,27,28,28,28,28,30,31,31,32,32,34,34,35,35,36,36,38,39,41,41,41,42,42,43,44,46,46,48,48,49,51,51,51,52,53,54,55,56,56,56,58,58,60,60,62,63,63,64,65,65,66,66,67,72,73,74,79,82,86,88,89,90,91,93,94,95,96,97,165,180,183,184]

In [ ]:
def getOnlyManualCheckData(params):
    pair_ids, source, destination, prs, cut_off_date = params
    start = time.perf_counter()
    
    ct = 0
    
    working_repos = []
    for pair in range(0, len(pair_ids)):
        print(pair_ids[pair], '-', source[pair], '-', destination[pair])
        common.verbose_mode = False

        try:
            destination_sha, ct = dataloader.getDestinationSha(destination[pair], cut_off_date[pair], token_list, ct)
            ct, pr_data, req, runtime = dataloader.fetchPrData(source[pair], destination[pair], prs[pair], destination_sha, token_list, ct)
            with open('Repos_prs/' + str(pair_ids[pair]) + '_' + source[pair].split('/')[0] + '_' + source[pair].split('/')[1] + '_prs.pkl', 'wb') as f:
                pickle.dump([pr_data, req, runtime], f)
            working_repos.append(pair_ids[pair])
        except Exception as e:
            print('Exception  :(\n ',e)
    print(working_repos)        
    finish = time.perf_counter()
    return f'Task done executing in ... {round((finish - start) / 60, 2)} minutes'

In [ ]:
def onlyManualCheckData():
    data = {}
    start_idx = 0

    for pair in pairs:
        if pair not in data:
            data[pair] = []
        data[pair].append(mc_prs[start_idx])        
        start_idx += 1
    
    
    source = []
    destination = []
    prs = []
    cut_off_date = []
    pair_ids =[]
    
    for i in data:
        pair_ids.append(i)
        source.append(dataset[i]['source'])
        destination.append(dataset[i]['destination'])
        
        prs_i = []
        for pr in data[i]:
            prs_i.append(str(pr))
        prs.append(prs_i)
        
        cut_off_date.append(dataset[i]['cut_off_date'])

    getOnlyManualCheckData((pair_ids, source, destination, prs, cut_off_date))
    
#     task = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
#     start1 = 0
#     end1 = 31
#     start2 = 31
#     end2 = 61
#     start3 = 61
#     end3 = 91
#     start4 = 91
#     end4 = 121
#     start5 = 121
#     end5 = 151
#     start6 = 151
#     end6 = 181
#     start7 = 181
#     end7 = 211
#     start8 = 211
#     end8 = 241
#     start9 = 241
#     end9 = 276
    
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         params = [
#                   (source, destination, prs, cut_off_date, task[0], start1, end1),
#                   (source, destination, prs, cut_off_date, task[1], start2, end2),
#                   (source, destination, prs, cut_off_date, task[2], start3, end3),
#                   (source, destination, prs, cut_off_date, task[3], start4, end4),
#                   (source, destination, prs, cut_off_date, task[4], start5, end5),
#                   (source, destination, prs, cut_off_date, task[5], start6, end6),
#                   (source, destination, prs, cut_off_date, task[6], start7, end7),
#                   (source, destination, prs, cut_off_date, task[7], start8, end8),
#                   (source, destination, prs, cut_off_date, task[8], start9, end9)
#                  ]
#         results = executor.map(getDataUsingThreads, params)
#         for result in results:
#             print(result)

In [ ]:
onlyManualCheckData()

In [ ]:
common.readResults(2, dataset[2]['source'])

# Checking the data

In [ ]:
to_exe = {'0': ['2185', '2479', '2484'],
 '10': ['8930', '10217'],
 '11': [],
 '12': [],
 '13': ['14350', '14391'],
 '14': ['232', '509', '655', '698'],
 '15': ['680', '684', '685', '690', '691', '692'],
 '16': ['913', '1000', '1047', '1060', '1071', '1195', '1215', '1231'],
 '17': ['760', '812'],
 '18': ['307', '318'],
 '19': ['760', '812', '855', '864', '884'],
 '1': ['14981', '15000', '15163', '15213', '15227', '15262', '15362'],
 '20': ['86', '203', '206', '237', '271', '284'],
 '21': ['30', '32', '33'],
 '22': ['1928', '2387', '2736', '2737'],
 '23': ['2739', '2742', '3497'],
 '24': ['63', '64'],
 '25': ['409', '773'],
 '26': ['574', '610', '809'],
 '27': ['62', '137', '161'],
 '28': ['258'],
 '29': ['235', '269'],
 '2': ['8697'],
 '30': ['184', '231'],
 '31': ['2276'],
 '32': ['173', '188'],
 '33': ['5758', '7069'],
 '34': ['1837', '1874'],
 '35': ['99', '138'],
 '36': [],
 '37': ['105', '153'],
 '38': ['117', '157'],
 '39': ['88', '105'],
 '3': ['1068', '1554', '1698'],
 '40': ['773', '779'],
 '41': ['1398', '1448'],
 '42': ['154', '203'],
 '43': ['199'],
 '44': ['159'],
 '45': ['7069'],
 '46': ['528'],
 '47': ['55'],
 '48': ['6077'],
 '49': ['352'],
 '4': ['7181', '7528', '7621', '7724', '8311', '8428'],
 '50': ['1024'],
 '51': ['1024'],
 '52': ['509'],
 '53': ['49'],
 '54': ['1414'],
 '55': ['128'],
 '56': ['611'],
 '57': [],
 '58': ['182'],
 '59': [],
 '5': [],
 '60': [],
 '61': ['528'],
 '62': ['5221'],
 '63': ['275'],
 '64': ['327'],
 '65': ['327'],
 '6': ['2344', '2345'],
 '7': ['4680', '4876', '4911', '4922', '4978', '5141'],
 '8': ['2320', '2333'],
 '9': ['7', '9', '14', '18']}

In [ ]:
ct =0
pair = '13'
source = dataset[int(pair)]['source']
destination = dataset[int(pair)]['destination']
cut_off_date = dataset[int(pair)]['cut_off_date']
prs = str(dataset[int(pair)]['pr']).split('/')
# prs = ['49']

destination_sha = ''
lenTokens = len(token_list)

if ct == lenTokens:
    ct = 0
cut_off_commits = commitloader.apiRequest('https://api.github.com/repos/' + destination +'/commits?until=' + cut_off_date, token_list[ct])
ct += 1

destination_sha = cut_off_commits[0]['sha']

# def fetchP|rData(source, destination, prs, destination_sha, token_list, ct):

In [ ]:
len(prs)

In [ ]:
from datetime import datetime

In [ ]:
destination_sha

In [ ]:
# 53-, 40-, 42-, 31, 36, 38, 39, 27, 28, 21-25

start = time.time()
req = 0
pr_data = {}
lenTokens = len(token_list)

for k in prs:
    if k not in to_exe[pair]:
        try:
            pr_data[k] = {}

    #         print(f'ct1 = {ct}')
            # Get the PR
            if ct == lenTokens:
                ct = 0


            pr_request = 'https://api.github.com/repos/' + source + '/pulls/' + k

            pr = commitloader.apiRequest(pr_request, token_list[ct])
            ct += 1
            req += 1

    #         print(f'ct2 = {ct}')
            # Get the commit
            if ct == lenTokens:
                ct = 0

#             print(pr)
            commits_url = pr['commits_url']
            commits = commitloader.apiRequest(commits_url, token_list[ct])
            common.verbose_print(f'ct ={ct}')
            ct += 1
            req += 1
    #         print(f'ct3 = {ct}')
    #         print('Commits\n', commits)
            commits_data = {}

            nr_files = pr['changed_files']

            pr_data[k]['pr_url'] = pr_request
            pr_data[k]['commits_url'] = commits_url
            pr_data[k]['changed_files'] = nr_files
            pr_data[k]['commits_data'] = list()
            pr_data[k]['destination_sha'] = destination_sha

            shas = []
            parents = []
            commits_dates = []

            for i in commits:
                if ct == lenTokens:
                    ct = 0
                commit_url = i['url']
                commit = commitloader.apiRequest(commit_url, token_list[ct])

                ct += 1
                req += 1
    #             print(f'ct4 = {ct}')

#                 print(commit)
                sha = commit['sha']
                if sha not in shas:
                    shas.append(sha)
                    parents.append(commit['parents'][0]['sha'])
                    commits_dates.append(datetime.strptime(commit['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ"))

                try:
                    files = commit['files']
                    for j in files:
                        status = j['status']
                        file_name = j['filename']
                        added_lines = j['additions']
                        removed_lines = j['deletions']
                        changes = j['changes']
                        file_ext = commitloader.get_file_type(file_name)

                        if file_name not in commits_data:
                            commits_data[file_name] = list()
                            if ct == lenTokens:
                                ct = 0

    #                         if file_name=='contrib/win32/win32compat/win32_usertoken_utils.c':
    #                             print(file_name)

                            if commitloader.findFile(file_name, destination, token_list[ct], destination_sha):
                                sub = {}
                                sub['commit_url'] = commit_url
                                sub['commit_sha'] = commit['sha']
                                sub['commit_date'] = commit['commit']['author']['date']
                                sub['parent_sha'] = commit['parents'][0]['sha']
                                sub['status'] =status
                                sub['additions'] = added_lines
                                sub['deletions'] = removed_lines
                                sub['changes'] = changes

                                commits_data[file_name].append(sub)


                            ct += 1
    #                         print(f'ct5 = {ct}')
                        else:
                            if ct == lenTokens:
                                ct = 0
                            if commitloader.findFile(file_name, destination, token_list[ct], destination_sha):
                                sub = {}
                                sub['commit_url'] = commit_url
                                sub['commit_sha'] = commit['sha']
                                sub['commit_date'] = commit['commit']['author']['date']
                                sub['parent_sha'] = commit['parents'][0]['sha']
                                sub['status'] =status
                                sub['additions'] = added_lines
                                sub['deletions'] = removed_lines
                                sub['changes'] = changes

                                commits_data[file_name].append(sub)

                            ct += 1
    #                         print(f'ct6 = {ct}')
                except Exception as e:
                    print(e)
                    print('This should only happen if there are no files changed in a commit')
            pr_data[k]['commits_data'].append(commits_data)

            min_date = min(commits_dates)
            max_date = max(commits_dates)
            min_index = commits_dates.index(min_date)
            max_index = commits_dates.index(max_date)
            pr_data[k]['first_commit_parent'] = parents[min_index]
            pr_data[k]['last_commit_sha'] = shas[max_index]

        except Exception as e:
            print('An error occurred while fetching the pull request information from GitHub.')
            print(pr_request)
            print (f'Error has to do with: {e}')

end = time.time()
runtime = end - start
print('Fetch Runtime: ', runtime)

with open('Repos_prs_opt/' + str(pair) + '_' + source.split('/')[0] + '_' + source.split('/')[1] + '_prs_to_exe.pkl', 'wb') as f:
    pickle.dump([pr_data, req, runtime], f)

In [ ]:
with open('Repos_prs_opt/' + str(pair) + '_' + source.split('/')[0] + '_' + source.split('/')[1] + '_prs_to_exe.pkl', 'wb') as f:
    pickle.dump([pr_data, req, runtime], f)

In [ ]:
len(pr_data)